# Conditional Independence
**Hands‑on Notebook**


**In this notebook**
Explore **conditional independence** in chain / fork / collider.



## 0) Setup

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(7)

def summarize_binary(y, x=None, do=None, df=None, name=""):
    if df is not None and x is not None:
        p = df.loc[df[x]==1, y].mean()
        n = (df[x]==1).sum()
        print(f"P({y}=1 | {x}=1) = {p:.3f}  [n={n}]  {name}")
    if df is not None and do is not None:
        p = df[y].mean()
        n = len(df)
        print(f"P({y}=1 | do({do})) = {p:.3f}  [n={n}]  {name}")



## A) Seeing vs Doing with the **Firing Squad** toy model

![Firing squad DAG](../images/firing_squad.png)

- Variables: 
`C` (captain order) 
`SA` (squad A fires)
`SB` (squad B fires)
`D` (death).  


We will compare:
- **Observation**: `P(D|SA=0)` — low, because when A doesn't fires, usually B also does not fire (same cause `C`).
- **Intervention**: `P(D|do(SA=0))` — set A to not fire regardless of the command `C`; isolate A's own causal contribution. Now we have some times B firind and some times B not firing resulting in a more complex scenario.


To have a bit of more interesting scenario, we impose some imperfection to obediance of our squads in the code below:

In [2]:
N = 100000
rng = np.random.default_rng(6)

# Structural equations (binary)
C = rng.binomial(1, 0.5, size=N)  # Captain order
SA = (C & (rng.random(N) < 0.95)).astype(int)  # Squad A obeys if ordered
SB = (C & (rng.random(N) < 0.95)).astype(int)  # Squad B obeys if ordered

# Lethality (set to 1 for simplicity, adjust if you want realism)
p_kill_A, p_kill_B = 1.0, 1.0
hit_A = p_kill_A
hit_B = p_kill_B
# hit_A = (SA & (rng.random(N) < p_kill_A)).astype(int)
# hit_B = (SB & (rng.random(N) < p_kill_B)).astype(int)
D = np.maximum(hit_A, hit_B)

# Combine all simulated variables into a single DataFrame for easier analysis and plotting
obs_df = pd.DataFrame(dict(C=C, SA=SA, SB=SB, D=D))

print("Observational world:")
# Compute P(D=1 | SA=0)
p_obs = obs_df.loc[obs_df["SA"] == 0, "D"].mean()
n_obs = (obs_df["SA"] == 0).sum()
print(f"P(D=1 | SA=0) = {p_obs:.3f}  [n={n_obs}]  (seeing)")

# --- Interventional: do(SA=0) ---
C2 = rng.binomial(1, 0.5, size=N)                   # captain as before
SA2 = np.zeros(N, dtype=int)                        # force A not to fire
SB2 = (C2 & (rng.random(N) < 0.95)).astype(int)     # B still reacts to captain

hit_A2 = (SA2 & (rng.random(N) < p_kill_A)).astype(int)
hit_B2 = (SB2 & (rng.random(N) < p_kill_B)).astype(int)
D2 = np.maximum(hit_A2, hit_B2)

do_df = pd.DataFrame(dict(C=C2, SA=SA2, SB=SB2, D=D2))
p_do = do_df["D"].mean()
n_do = len(do_df)

print("\nInterventional world:")
print(f"P(D=1 | do(SA=0)) = {p_do:.3f}  [n={n_do}]  (doing)")


Observational world:
P(D=1 | SA=0) = 1.000  [n=52415]  (seeing)

Interventional world:
P(D=1 | do(SA=0)) = 0.475  [n=100000]  (doing)


Number n above shows the number of tests where SA=0 happened. When we only observed, about half of the time, SA=0 and when we intervened, it was always kept at 0.

#### We see that P(D=1 | SA=0) ≠ P(D=1 | do(SA=0))!
This difference reveals that **C (the captain’s order)** is a **confounder** — it influences both the squad’s action (`SA`) and the outcome (`D`).


## B) Conditional Independence in **Chain / Fork / Collider**

In this section we simulate three fundamental causal structures (often called the *building blocks* of causal graphs)  
to explore how **conditional independence** behaves in each.

Reminder:
### Marginal vs Conditional Correlation

- **Marginal correlation** measures how two variables vary together *overall*, without taking any other variables into account.  
  → Example: the raw relationship between Smoking and Cancer in the population.

- **Conditional correlation** measures how two variables relate *after we fix or control for* a third variable.  
  → Example: the relationship between Smoking and Cancer **within each level of Tar exposure**.

**Key idea:**  
If two variables are correlated marginally but not conditionally, it means a third variable (a mediator or confounder) explains their association.  
Conversely, if they are independent marginally but correlated conditionally, conditioning has **opened a path** (as in collider bias).

---

### 1. Chain: A → B → C
**Interpretation:**  
B, "the mediator" transmits information or influence from A to C.  
- *Example:* Smoking → Tar in lungs → Cancer.  
- A and C are correlated because information “flows” through B.  
- **If we condition on B**, we block that path — A and C become (approximately) independent.

**Expectation:**  
- Marginal correlation: high (A and C move together).  
- Conditional correlation given B: ≈ 0 (path blocked).

---

### 2. Fork (Confounding): A ← U → C
**Interpretation:**  
U is a *common cause* (confounder) of both A and C.  
- *Example:* Genetic predisposition → Smoking and Cancer.  
- A and C appear correlated, but only because of U.  
- **If we condition on U**, we remove that shared cause and eliminate the spurious correlation.

**Expectation:**  
- Marginal correlation: high (U induces a false link).  
- Conditional correlation given U: ≈ 0 (confounding removed).

---

### 3. Collider: A → B ← C
**Interpretation:**  
B is a *common effect* (collider) of A and C.  
- *Example:*  
  - A = Smoking  
  - C = Air pollution  
  - B = Hospital admission (caused by either).  
- Normally, A and C are independent.  
- **If we condition on B** (or any descendant of B), we *create* a correlation between A and C —  
  this is known as **collider bias** or **selection bias**.

**Expectation:**  
- Marginal correlation: near 0 (A, C independent).  
- Conditional correlation given B: strong (conditioning opens the path).

---


### Conclusion
> Correlation alone can mislead: depending on the graph, conditioning can remove, reveal, or even **fabricate** relationships.
>
> Understanding which paths are open or closed (via **d-separation**) is central to causal inference.



### What the following block of code does
- Each function (`sim_chain`, `sim_fork`, `sim_collider`) simulates random data following these causal relationships.  
- We compute:
  - **Marginal correlation**: `corr(A, C)`  
  - **Conditional correlation**: `corr(A, C | middle node)` using simple binning on the conditioning variable.
- This shows how *conditioning* can either **block** or **create** associations depending on the graph structure.


In [3]:

def sim_chain(N=50_000, seed=1):
    rng = np.random.default_rng(seed)
    A = rng.normal(0,1,N)
    B = A + rng.normal(0,1,N)
    C = B + rng.normal(0,1,N)
    return pd.DataFrame(dict(A=A,B=B,C=C))

def sim_fork(N=50_000, seed=2):
    rng = np.random.default_rng(seed)
    U = rng.normal(0,1,N)
    A = U + rng.normal(0,1,N)
    C = U + rng.normal(0,1,N)
    return pd.DataFrame(dict(U=U,A=A,C=C))

def sim_collider(N=50_000, seed=3):
    rng = np.random.default_rng(seed)
    A = rng.normal(0,1,N)
    C = rng.normal(0,1,N)
    B = A + C + rng.normal(0,1,N)  # collider
    return pd.DataFrame(dict(A=A,B=B,C=C))

def corr(x,y,df):
    return np.corrcoef(df[x], df[y])[0,1]

chain = sim_chain()
fork = sim_fork()
coll = sim_collider()

print("Chain: corr(A,C)  (marginal) =", corr("A","C", chain))
print("Fork:  corr(A,C)  (marginal) =", corr("A","C", fork))
print("Collider: corr(A,C) (marginal) =", corr("A","C", coll))

# Conditioning effects
def partial_corr_xy_given_z(x,y,z,df, bins=10):
    # Approximate partial correlation by binning on z (simple classroom-friendly approach).
    df2 = df.copy()
    df2["_zb"] = pd.qcut(df2[z], q=bins, duplicates="drop")
    vals = []
    for _,grp in df2.groupby("_zb", observed=True):
        if len(grp)>5:
            vals.append(np.corrcoef(grp[x], grp[y])[0,1])
    return np.nanmean(vals)

print("\nConditioning (approx via binning):")
print("Chain: corr(A,C | B) ≈", partial_corr_xy_given_z("A","C","B", chain))
print("Fork:  corr(A,C | U) ≈", partial_corr_xy_given_z("A","C","U", fork))
print("Collider: corr(A,C | B) ≈", partial_corr_xy_given_z("A","C","B", coll))


Chain: corr(A,C)  (marginal) = 0.5801684286913067
Fork:  corr(A,C)  (marginal) = 0.49713950788544
Collider: corr(A,C) (marginal) = -0.01064898968845041

Conditioning (approx via binning):
Chain: corr(A,C | B) ≈ 0.047197088807209964
Fork:  corr(A,C | U) ≈ 0.03878851215357192
Collider: corr(A,C | B) ≈ -0.4745972035238439


### Interpretation of Results

| Structure | Marginal Corr(A, C) | Conditional Corr(A, C \| Z) | What it shows |
|------------|--------------------:|-----------------------------:|----------------|
| **Chain** | 0.58 | 0.05 | Conditioning on the mediator **B** blocks the flow from A → B → C. |
| **Fork** | 0.50 | 0.04 | Conditioning on the confounder **U** removes the common-cause association. |
| **Collider** | −0.01 | −0.47 | Conditioning on **B** (a common effect) creates a spurious link — classic *collider bias*. |

**Summary:**  
- **Chain & Fork:** conditioning *reduces* correlation (closes the path).  
- **Collider:** conditioning *induces* correlation (opens a blocked path).


In [ ]:
#Is a collider correlation always a negative value? Was this just a coincidence with the random seed?


## C) **Proxy variable** for an unobserved confounder

Unobserved `U` (true smoking exposure) affects both `YellowTeeth (Z)` and `Cancer (Y)`;  
`Smoking (X)` is noisy self-report we can't rely on for percision issues. Nicotin level in body measured accurately `NL` serves as a **proxy** for `U`.

We compare naive estimate `P(Y|X)` with adjustment by the proxy `NL` (back-door via proxy).


In [4]:
# --- Section C (revised): Proxy variable with accurate biomarker NL ---

N = 200_000
rng = np.random.default_rng(12)

# Unobserved true exposure
U = rng.normal(0, 1, N)                 # unobserved driver of risk

# Observed variables
X  = U + rng.normal(0, 1.0, N)          # self-report (noisy, low precision)
Z  = U + rng.normal(0, 0.8, N)          # yellow teeth (crude indicator; we won't use it for adjustment here)
NL = U + rng.normal(0, 0.1, N)          # biomarker (accurate proxy; low noise)

# Outcome depends on TRUE exposure (U), not X directly
logit = -0.7 + 1.4 * U
pY = 1 / (1 + np.exp(-logit))
Y = (rng.random(N) < pY).astype(int)

dfp = pd.DataFrame(dict(X=X, Z=Z, NL=NL, Y=Y))

# --- Models: naive vs proxy-adjusted ---
import statsmodels.api as sm

# 1) Naive: Y ~ X  (confounded by U)
m_naive = sm.Logit(dfp["Y"], sm.add_constant(dfp[["X"]])).fit(disp=False)

# 2) Proxy-adjusted with accurate biomarker: Y ~ X + NL
m_proxy = sm.Logit(dfp["Y"], sm.add_constant(dfp[["X","NL"]])).fit(disp=False)

# 3) Biomarker only: Y ~ NL  (close to the "oracle" using U)
m_biomarker = sm.Logit(dfp["Y"], sm.add_constant(dfp[["NL"]])).fit(disp=False)

print("Predicting Y (cancer) using self reported smoking only")
print("Naive (Y ~ X):")
print(f"  beta_X = {m_naive.params['X']:.3f}")

print("\nProxy-adjusted to include both self report and Nicotin level biomarker(Y ~ X + NL):")
print(f"  beta_X  = {m_proxy.params['X']:.3f}   (should shrink toward 0)")
print(f"  beta_NL = {m_proxy.params['NL']:.3f}  (captures the true U effect)")

print("\nPredicting Y (cancer) using Biomarker only (Y ~ NL):")
print(f"  beta_NL = {m_biomarker.params['NL']:.3f}")

# (Optional instructor check — uncomment to peek at "truth")
# corr_U_X  = np.corrcoef(U, X)[0,1]
# corr_U_Z  = np.corrcoef(U, Z)[0,1]
# corr_U_NL = np.corrcoef(U, NL)[0,1]
# print(f"\n[Hidden truth] corr(U,X)={corr_U_X:.2f}, corr(U,Z)={corr_U_Z:.2f}, corr(U,NL)={corr_U_NL:.2f}")


Predicting Y (cancer) using self reported smoking only
Naive (Y ~ X):
  beta_X = 0.579

Proxy-adjusted to include both self report and Nicotin level biomarker(Y ~ X + NL):
  beta_X  = 0.006   (should shrink toward 0)
  beta_NL = 1.356  (captures the true U effect)

Predicting Y (cancer) using Biomarker only (Y ~ NL):
  beta_NL = 1.361



> **Observation:** With only `X` we pick up confounding from `U`.  
> Adding the proxy `Z` absorbs much of `U`'s influence and moves `beta_X` toward the *direct* effect.


## Excersice:

Secion A) In the parameterization `P(D|SA=0)` and `P(D|do(SA=0))` can both be close to 1.  
What *qualitatively* changes between the two worlds? Explain using a one-sentence reference to the DAG.


In [ ]:
#In this scenario, for both P(D|SA=0) and P(D|do(SA=0) to be close to 1, we would need to 
#ensure that Squad B will always follow the Captain's order and has a lethality of 1.0 as well.


## E) Quick Tasks (for credit / discussion)

1. **Parameter flip:** In the firing squad model, change `p_kill_A` to 0.6 and `p_kill_B` to 0.99.  
   - Re-run and record `P(D|SA=1)` vs `P(D|do(SA=1))`.  
   - Explain in one sentence which way confounding moves the observational estimate.

2. **Collider bias:** In section B, filter to the top 10% of `B` values in the collider model and compute `corr(A,C)` there.  
   - Why does this selection amplify the association?

3. **Proxy strength:** In section C, increase proxy noise (e.g., `Z = U + 1.5*eps_z`).  
   - How do `beta_X` and `beta_Z` change? What does this say about **weak proxies**?

4. **Back-door bins:** In section D, increase the number of `U` bins from 10 to 30.  
   - Does the adjusted estimate stabilize? Why / why not?

> *Tip:* Add a new cell under each section to keep your answers and code together.


In [ ]:
#Quick Task 1

In [5]:
N = 100000
rng = np.random.default_rng(6)

# Structural equations (binary)
C = rng.binomial(1, 0.5, size=N)  # Captain order
SA = (C & (rng.random(N) < 0.95)).astype(int)  # Squad A obeys if ordered
SB = (C & (rng.random(N) < 0.95)).astype(int)  # Squad B obeys if ordered

# Lethality (set to 1 for simplicity, adjust if you want realism)
p_kill_A, p_kill_B = 0.6, 0.99
hit_A = p_kill_A
hit_B = p_kill_B
# hit_A = (SA & (rng.random(N) < p_kill_A)).astype(int)
# hit_B = (SB & (rng.random(N) < p_kill_B)).astype(int)
D = np.maximum(hit_A, hit_B)

# Combine all simulated variables into a single DataFrame for easier analysis and plotting
obs_df = pd.DataFrame(dict(C=C, SA=SA, SB=SB, D=D))

print("Observational world:")
# Compute P(D=1 | SA=0)
p_obs = obs_df.loc[obs_df["SA"] == 0, "D"].mean()
n_obs = (obs_df["SA"] == 0).sum()
print(f"P(D=1 | SA=0) = {p_obs:.3f}  [n={n_obs}]  (seeing)")

# --- Interventional: do(SA=0) ---
C2 = rng.binomial(1, 0.5, size=N)                   # captain as before
SA2 = np.zeros(N, dtype=int)                        # force A not to fire
SB2 = (C2 & (rng.random(N) < 0.95)).astype(int)     # B still reacts to captain

hit_A2 = (SA2 & (rng.random(N) < p_kill_A)).astype(int)
hit_B2 = (SB2 & (rng.random(N) < p_kill_B)).astype(int)
D2 = np.maximum(hit_A2, hit_B2)

do_df = pd.DataFrame(dict(C=C2, SA=SA2, SB=SB2, D=D2))
p_do = do_df["D"].mean()
n_do = len(do_df)

print("\nInterventional world:")
print(f"P(D=1 | do(SA=0)) = {p_do:.3f}  [n={n_do}]  (doing)")

Observational world:
P(D=1 | SA=0) = 0.990  [n=52415]  (seeing)

Interventional world:
P(D=1 | do(SA=0)) = 0.470  [n=100000]  (doing)


In [ ]:
#Since we changed the lethality probabilities, we see that given Squad A does not fire, 
#the probability of death is directly related to the lethality of Squad B.

In [ ]:
#Quick Task 2

In [17]:

def sim_chain(N=50_000, seed=1):
    rng = np.random.default_rng(seed)
    A = rng.normal(0,1,N)
    B = A + rng.normal(0,1,N)
    C = B + rng.normal(0,1,N)
    return pd.DataFrame(dict(A=A,B=B,C=C))

def sim_fork(N=50_000, seed=2):
    rng = np.random.default_rng(seed)
    U = rng.normal(0,1,N)
    A = U + rng.normal(0,1,N)
    C = U + rng.normal(0,1,N)
    return pd.DataFrame(dict(U=U,A=A,C=C))

def sim_collider(N=50_000, seed=3):
    rng = np.random.default_rng(seed)
    A = rng.normal(0,1,N)
    C = rng.normal(0,1,N)
    B = (A + C + rng.normal(0,1,N) * .1)  # collider
    return pd.DataFrame(dict(A=A,B=B,C=C))

def corr(x,y,df):
    return np.corrcoef(df[x], df[y])[0,1]

chain = sim_chain()
fork = sim_fork()
coll = sim_collider()

print("Chain: corr(A,C)  (marginal) =", corr("A","C", chain))
print("Fork:  corr(A,C)  (marginal) =", corr("A","C", fork))
print("Collider: corr(A,C) (marginal) =", corr("A","C", coll))

# Conditioning effects
def partial_corr_xy_given_z(x,y,z,df, bins=10):
    # Approximate partial correlation by binning on z (simple classroom-friendly approach).
    df2 = df.copy()
    df2["_zb"] = pd.qcut(df2[z], q=bins, duplicates="drop")
    vals = []
    for _,grp in df2.groupby("_zb", observed=True):
        if len(grp)>5:
            vals.append(np.corrcoef(grp[x], grp[y])[0,1])
    return np.nanmean(vals)

print("\nConditioning (approx via binning):")
print("Chain: corr(A,C | B) ≈", partial_corr_xy_given_z("A","C","B", chain))
print("Fork:  corr(A,C | U) ≈", partial_corr_xy_given_z("A","C","U", fork))
print("Collider: corr(A,C | B) ≈", partial_corr_xy_given_z("A","C","B", coll))

Chain: corr(A,C)  (marginal) = 0.5801684286913067
Fork:  corr(A,C)  (marginal) = 0.49713950788544
Collider: corr(A,C) (marginal) = -0.01064898968845041

Conditioning (approx via binning):
Chain: corr(A,C | B) ≈ 0.047197088807209964
Fork:  corr(A,C | U) ≈ 0.03878851215357192
Collider: corr(A,C | B) ≈ -0.9205779788781971


In [ ]:
#corr(A,C | B) is amplified in this case because we are taking a much smaller sample size 
#of data that is conditioned on B, which creates a stronger observed correlation between A and C.

In [ ]:
#Quick Task 3